In [1]:
import os
from pathlib import Path
import DifferentiableStateSpaceModels
from DifferentiableStateSpaceModels import *
from sympy import *
import numpy as np
import pandas as pd

init_printing()  # for nice sympy expressions in jupyter

In [2]:
# Define the symbols
α, β, ρ, δ, σ, Ω_1 = symbols('α β ρ δ σ Ω_1')
t = symbols('t', integer=True)
k, z, c, q = symbols('k z c q', cls=Function)

# Define the states, controls, and parameters
x = [k, z]  # states
y = [c, q]  # controls
p = [α, β, ρ, δ, σ, Ω_1]  # parameters

# Define the system of model equations
H = [
    1 / c(t) - (β / c(t + 1)) * (α * exp(z(t + 1)) * k(t + 1)**(α - 1) + (1 - δ)),
    c(t) + k(t + 1) - (1 - δ) * k(t) - q(t),
    q(t) - exp(z(t)) * k(t)**α,
    z(t + 1) - ρ * z(t)
]

# Define the steady states
steady_states = [
    Eq(k(oo), (((1 / β) - 1 + δ) / α)**(1 / (α - 1))),
    Eq(z(oo), 0),
    Eq(c(oo), (((1 / β) - 1 + δ) / α)**(α / (α - 1)) - δ * (((1 / β) - 1 + δ) / α)**(1 / (α - 1))),
    Eq(q(oo), (((1 / β) - 1 + δ) / α)**(α / (α - 1)))
]

# Define the matrix for the 1 shock
Γ = Matrix([σ])

# Define the n_x * n_ϵ matrix
η = Matrix([0, -1])

# Define the observation matrix
Q = Matrix([[1.0, 0, 0, 0], [0, 0, 1.0, 0]])

# Define the diagonal cholesky of covariance matrix for observation noise
Ω = [Ω_1, Ω_1]

# Generates the files and includes if required.  If the model is already created, then just loads
overwrite_model_cache = True

In [3]:
def default_model_cache_location():
    """
    Returns the default location for the model cache
    :return: path of a folder joining the package directory and ".function_cache"
    """
    return os.path.join(Path(DifferentiableStateSpaceModels.__file__).parent.absolute(), ".function_cache")


model_cache_location = default_model_cache_location()
max_order = 2
steady_states_iv = None
model_name = "rbc_notebook_example"
model_cache_location = default_model_cache_location()
overwrite_model_cache = False
print_level = 1
max_order = 2
save_ip = True
save_oop = False  # only does inplace by default
skipzeros = True
fillzeros = False
simplify_Ψ = True
simplify = True
simplify_p = True

In [4]:
# path of the module to be saved in the cache
module_cache_path = os.path.join(model_cache_location, model_name + ".py")

print_level = 1
# only load cache if the module isn't already loaded in memory
if (str(model_name) in globals()) and (not overwrite_model_cache):
    if print_level > 0:
        print(f"Using existing module {model_name}\n")

# if path already exists
if (os.path.exists(module_cache_path)) and (not overwrite_model_cache):
    # path exists and not overwriting
    if print_level > 0:
        print(f"Model already generated at {module_cache_path}\n")

In [5]:
    n_y = len(y)
    n_x = len(x)
    n = n_y + n_x
    n_p = len(p)
    assert n_p > 0  # code written to have at least one parameter
    n_ϵ = η.shape[1]
    n_z = n if Q is None else Q.shape[0]

In [6]:
    # Get the markovian variables and create substitutions
    y_subs = [make_substitutions(t, y_i) for y_i in y]
    x_subs = [make_substitutions(t, x_i) for x_i in x]
    y = Matrix([y_i['var'] for y_i in y_subs])
    x = Matrix([x_i['var'] for x_i in x_subs])
    y_p = Matrix([y_i['var_p'] for y_i in y_subs])
    x_p = Matrix([x_i['var_p'] for x_i in x_subs])
    y_ss = Matrix([y_i['var_ss'] for y_i in y_subs])
    x_ss = Matrix([x_i['var_ss'] for x_i in x_subs])
    subs = x_subs + y_subs
    all_to_markov = [sub['markov_t'] for sub in subs] + [sub['markov_tp1'] for sub in subs] + [sub['markov_inf'] for sub in subs]
    all_to_var = [sub['tp1_to_var'] for sub in subs] + [sub['inf_to_var'] for sub in subs]

In [7]:
def equations_to_dict(equations):
    return {str(eq.lhs.subs(all_to_markov).subs(all_to_var)): eq.rhs.subs(all_to_markov) for eq in equations}

In [8]:
    if print_level > 0:
        print("\033[96mBuilding model up to order {}\033[0m".format(max_order))
    if print_level > 1:
        print("\033[96msimplify = {}, simplify_p = {}, simplify Ψ = {}\033[0m".format(simplify, simplify_p, simplify_Ψ))

Building model up to order 2


In [9]:
# create functions in correct order
y_bar = None if steady_states is None else order_vector_by_symbols(
    equations_to_dict(steady_states), [y_sub['symbol'] for y_sub in y_subs])
x_bar = None if steady_states is None else order_vector_by_symbols(
    equations_to_dict(steady_states), [x_sub['symbol'] for x_sub in x_subs])
y_bar_iv = None if steady_states_iv is None else order_vector_by_symbols(
    equations_to_dict(steady_states_iv), [y_sub['symbol'] for y_sub in y_subs])
x_bar_iv = None if steady_states_iv is None else order_vector_by_symbols(
    equations_to_dict(steady_states_iv), [x_sub['symbol'] for x_sub in x_subs])

In [12]:
# Get any latex generated stuff we wish for pretty display of the model
H_latex = latex(H)
steady_states_latex = latex(steady_states)
steady_states_iv_latex = latex(steady_states_iv)

In [15]:
print(steady_states_latex)

\left[ k{\left(\infty \right)} = \left(\frac{δ - 1 + \frac{1}{β}}{α}\right)^{\frac{1}{α - 1}}, \  z{\left(\infty \right)} = 0, \  c{\left(\infty \right)} = - δ \left(\frac{δ - 1 + \frac{1}{β}}{α}\right)^{\frac{1}{α - 1}} + \left(\frac{δ - 1 + \frac{1}{β}}{α}\right)^{\frac{α}{α - 1}}, \  q{\left(\infty \right)} = \left(\frac{δ - 1 + \frac{1}{β}}{α}\right)^{\frac{α}{α - 1}}\right]


$$\left[ k{\left(\infty \right)} = \left(\frac{δ - 1 + \frac{1}{β}}{α}\right)^{\frac{1}{α - 1}}, \  z{\left(\infty \right)} = 0, \  c{\left(\infty \right)} = - δ \left(\frac{δ - 1 + \frac{1}{β}}{α}\right)^{\frac{1}{α - 1}} + \left(\frac{δ - 1 + \frac{1}{β}}{α}\right)^{\frac{α}{α - 1}}, \  q{\left(\infty \right)} = \left(\frac{δ - 1 + \frac{1}{β}}{α}\right)^{\frac{α}{α - 1}}\right]$$